In [1]:
import numpy as np
import qutip as qt
from scipy.linalg import logm, expm
from qiskit.quantum_info import Operator, state_fidelity
from qiskit import QuantumCircuit
from copy import deepcopy

from tools.classical import *
from tools.quantum import *

#### Qutip Exact Hamiltonian

In [15]:
num_qubits = 3

X = qt.sigmax()
Y = qt.sigmay()
Z = qt.sigmaz()
coeffs = [1, 1, 1]
hamiltonian_qt = hamiltonian_matrix([X, X], [Y, Y], [Z, Z], coeffs=coeffs, num_qubits=num_qubits, symbreak_term=[Z])
# print(hamiltonian_qt)
#* HAMILTONIANS
exact_spec = np.linalg.eigvalsh(hamiltonian_qt)
rescaling_factor, shift = rescaling_and_shift_factors(hamiltonian_qt)  # Shift already rescaled
rescaled_hamiltonian_qt = hamiltonian_qt / rescaling_factor + shift * qt.qeye(hamiltonian_qt.shape[0])

#* SPECTRUMS
# print(f'Shifted spectrum {np.linalg.eigvalsh(shifted_hamiltonian_qt)}')
print(f'Rescaling factor {rescaling_factor}')
print(f'Shift {shift}')
print(f'Original spectrum {exact_spec}')
print(f'Normalized spectrum {np.linalg.eigvalsh(rescaled_hamiltonian_qt)}')

Rescaling factor 8.0
Shift 0.5
Original spectrum [-4.         -3.46410162 -2.82842712 -2.          2.          2.82842712
  3.46410162  4.        ]
Normalized spectrum [-7.11644431e-17  6.69872981e-02  1.46446609e-01  2.50000000e-01
  7.50000000e-01  8.53553391e-01  9.33012702e-01  1.00000000e+00]


### Qutip unitary evolution construction and comparison: Exact vs Trotterized

$n = 3, k = 4$
Hamiltonian is rescaled already by its coupling strength, not by the rescaled time anymore (bit easier to have an overview)
$$H = 0.125\times Heisenberg_3 + Z_{bulk}$$

In [16]:
T = 1
total_time = T * 2 * np.pi
num_trotter_steps = 10
step_size = total_time / num_trotter_steps

rescaled_coeffs = coeffs / rescaling_factor

#* EXACT UNITARIES
exact_U_qt = expm(1j * rescaled_hamiltonian_qt * total_time)

#* QUTIP TROTTER
trott_U_qt = trotter_heisenberg_qutip(num_qubits, step_size, num_trotter_steps, coeffs=rescaled_coeffs, shift=(total_time*shift))
# UUdag_minus_id = exact_U_qt * trott_U_qt.dag() - qt.qeye(exact_U_qt.shape[0])

#* QUTIP DISTANCES
dist_between_unitaries = np.linalg.norm(exact_U_qt - trott_U_qt)
print(f'Distance between Qutip Trotter vs Exact {dist_between_unitaries}')

# with np.printoptions(precision=3, suppress=True):
#     print(exact_U_qt)
#     print(trott_U_qt)
#     print(UUdag_minus_id)
    
# logU_shifted = logm(shifted_trott_U_qt) / (1j * T)
# logU = logm(trott_U_qt) / (1j * T)
# dist_shifted = np.linalg.norm(logU_shifted - shifted_exact_U_qt)
# dist = np.linalg.norm(logU - exact_U_qt)
# print(dist_shifted)
# print(dist)

Distance between Qutip Trotter vs Exact 3.6867911378274805


### Observable expectation comparison: Exact vs Trotterized, finding a good number of Trotter steps

In [14]:
#* Initial state
# initial_state = np.ones(2**num_qubits) / np.sqrt(2**num_qubits)  # Equal superposition
np.random.seed(667)
randstate_better = np.zeros(2**num_qubits)
randstate_better[np.random.choice(2**num_qubits, 2**num_qubits//4, replace=False)] = 1
randstate_better /= np.linalg.norm(randstate_better)
rand_ini_state = randstate_better
rand_ini_state_qt = rand_ini_state[::-1]  # Qutip uses reverse order

initial_energy = qt.expect(qt.Qobj(hamiltonian_qt), qt.Qobj(rand_ini_state_qt))
print(f'Initial energy {initial_energy}')

# Pretty garbage:
# rand_state0 = np.random.random(2**num_qubits) + 1j * np.random.random(2**num_qubits)  # Random state
# rand_state0 /= np.linalg.norm(rand_state0)
# rand_state1 = np.random.random(2**num_qubits) + 1j * np.random.random(2**num_qubits)  # Random state
# rand_state1 /= np.linalg.norm(rand_state1)
# print(qt.fidelity(qt.Qobj(randstate_better), qt.Qobj(rand_state1)))

#* Evolved states
exact_evolved_state = qt.Qobj(exact_U_qt) * qt.Qobj(rand_ini_state_qt)
trott_evolved_state = qt.Qobj(trott_U_qt) * qt.Qobj(rand_ini_state_qt)

exact_evolved_state_shifted = qt.Qobj(shifted_exact_U_qt) * qt.Qobj(rand_ini_state_qt)
trott_evolved_state_shifted = qt.Qobj(shifted_trott_U_qt) * qt.Qobj(rand_ini_state_qt)

exact_evolved_state_normalized = qt.Qobj(normalized_exact_U_qt) * qt.Qobj(rand_ini_state_qt)
trott_evolved_state_normalized = qt.Qobj(normalized_trott_U_qt) * qt.Qobj(rand_ini_state_qt)

#* Fidelities
exact_evolved_fidelity = qt.fidelity(exact_evolved_state, trott_evolved_state)
exact_evolved_fidelity_normalized = qt.fidelity(exact_evolved_state_normalized, trott_evolved_state_normalized)
# print('Shifted:')  #* Global phase won't show up in fidelity
# fid_of_states_shifted = qt.fidelity(exact_evolved_state_shifted, trott_evolved_state_shifted)
print(f'Exact-Evolved fidelity {exact_evolved_fidelity}')
print(f'Rescaled + shifted exact-evolved fidelity {exact_evolved_fidelity_normalized}')


Initial energy 1.4999999999999998
Exact-Evolved fidelity 0.6144372666330262
Rescaled + shifted exact-evolved fidelity 0.9949855369573022


#### Circuit

In [15]:
trotter_step = trotter_step_heisenberg(num_qubits)
ham_time_evol = lambda m, T, sh: ham_evol(num_qubits, trotter_step=trotter_step, num_trotter_steps=m, time=T,
                                          shift = sh)

# num_trott_steps = int(np.ceil(8 * np.pi * np.abs(T) / rescaling_factor))

time_evol_circ = ham_time_evol(num_trotter_steps, total_time, 0)
time_evol_circ_shifted = ham_time_evol(num_trotter_steps, total_time, total_time*just_shift)
time_evol_circ_normalized = ham_time_evol(num_trotter_steps, rescaled_time, total_time*shift)

circ = QuantumCircuit(num_qubits, name="H")
circ_shifted = QuantumCircuit(num_qubits, name="H")
circ_normalized = QuantumCircuit(num_qubits, name="H")

circ.compose(time_evol_circ, range(num_qubits), inplace=True)
circ_shifted.compose(time_evol_circ_shifted, range(num_qubits), inplace=True)
circ_normalized.compose(time_evol_circ_normalized, range(num_qubits), inplace=True)

print(f'#Qubits {num_qubits}')
print(f'#Trotter steps {num_trotter_steps}')
print(f'#Gates {len([(op, _, _) for (op, _, _) in circ.data])}')
# print(circ)
# print(circ_shifted)
print(circ_normalized)


#Qubits 5
#Trotter steps 10
#Gates 150
global phase: 3.7643
     ┌────────────────┐┌────────────────┐                                 »
q_0: ┤0               ├┤0               ├─■───────────────────────────────»
     │  Rxx(-0.10076) ││  Ryy(-0.10076) │ │ZZ(-0.10076) ┌────────────────┐»
q_1: ┤1               ├┤1               ├─■─────────────┤0               ├»
     └────────────────┘└────────────────┘               │  Rxx(-0.10076) │»
q_2: ───────────────────────────────────────────────────┤1               ├»
                                                        └────────────────┘»
q_3: ─────────────────────────────────────────────────────────────────────»
                                                                          »
q_4: ─────────────────────────────────────────────────────────────────────»
                                                                          »
«                                                                          »
«q_0: ─────────────────────

### Comparing Circuit unitary operator to Exact

In [16]:
#* Circuit operators
trotterized_evolution_unitary = Operator(circ).data
trotterized_evolution_unitary_shifted = Operator(circ_shifted).data
trotterized_evolution_unitary_normalized = Operator(circ_normalized).data

#* Distance to exact unitary
dist_circ_unitary_vs_exact = np.linalg.norm(trotterized_evolution_unitary - exact_U_qt)
dist_circ_unitary_vs_exact_shifted = np.linalg.norm(trotterized_evolution_unitary_shifted - shifted_exact_U_qt)
dist_circ_unitary_vs_exact_normalized = np.linalg.norm(trotterized_evolution_unitary_normalized - normalized_exact_U_qt)

print(f'Distance between circ unitary and exact {dist_circ_unitary_vs_exact}')
print(f'Distance between circ unitary and exact shifted {dist_circ_unitary_vs_exact_shifted}')
print(f'Distance between circ unitary and exact normalized {dist_circ_unitary_vs_exact_normalized}')

Distance between circ unitary and exact 6.33921232241323
Distance between circ unitary and exact shifted 6.339212322413235
Distance between circ unitary and exact normalized 0.6377098583002071


### Fidelity of Circuit vs Exact

In [22]:
circ_with_ini_state = QuantumCircuit(num_qubits, name="H")
circ_with_ini_state_normalized = QuantumCircuit(num_qubits, name="H")

circ_with_ini_state.initialize(rand_ini_state, range(num_qubits))
circ_with_ini_state_normalized.initialize(rand_ini_state, range(num_qubits))

circ_with_ini_state.compose(time_evol_circ, range(num_qubits), inplace=True)
circ_with_ini_state_normalized.compose(time_evol_circ_normalized, range(num_qubits), inplace=True)

qiskit_end_state = Statevector(circ_with_ini_state)  # Circ to state
qiskit_end_state_normalized = Statevector(circ_with_ini_state_normalized)

# qiskit_end_state = qt.Qobj(Statevector(circ_with_ini_state).data[::-1])  # Circ to state
# qiskit_end_state_normalized = qt.Qobj(Statevector(circ_with_ini_state_normalized).data[::-1])
# qiskit_end_state_2 = qt.Qobj(trotterized_evolution_unitary) * qt.Qobj(rand_ini_state)  # Circ to op, op * ini_state

print('QISKIT FIDELITY')
qiskit_trott_evolved_state = Statevector(trott_evolved_state.full()[::-1])
qiskit_trott_evolved_state_normalized = Statevector(trott_evolved_state_normalized.full()[::-1])
print('EXACT')
print(f'Fidelity Trotters {state_fidelity(qiskit_trott_evolved_state, qiskit_end_state)}')
print('NORMALIZED')
print(f'Fidelity Trotters {state_fidelity(qiskit_trott_evolved_state_normalized, qiskit_end_state_normalized)}')



#TODO: Check Qiskit vs Qutip convention for up - down, i.e. MSB - LSB
#TODO: 1 more or less Trotter steps than Qutip
# print('EXACT')
# print(f'Fidelity Trotters {qt.fidelity(trott_evolved_state, qiskit_end_state)}')
# print(f'Fidelity Qutip Trotter vs Exact {qt.fidelity(trott_evolved_state, exact_evolved_state)}')
# print(f'Fidelity Qiskit Trotter vs Exact {qt.fidelity(qiskit_end_state, exact_evolved_state)}')

# print('NORMALIZED')
# print(f'Fidelity Trotters {qt.fidelity(trott_evolved_state_normalized, qiskit_end_state_normalized)}')
# print(f'Fidelity Qutip Trotter vs Exact {qt.fidelity(trott_evolved_state_normalized, exact_evolved_state_normalized)}')
# print(f'Fidelity Qiskit Trotter vs Exact {qt.fidelity(qiskit_end_state_normalized, exact_evolved_state_normalized)}')

QISKIT FIDELITY
EXACT
Fidelity Trotters 0.4764764344819928
NORMALIZED
Fidelity Trotters 0.9595826842104977


### Trotterized Hamiltonian comparison, Qiskit vs Exact, Qutip vs Exact

In [18]:
#* LOGM
# logU = logm(trotterized_evolution_unitary)  # log U = i 2pi H T / rescaling_factor  #! It's not unique if any eigenenergy < 0
# trotterized_ham_from_circuit = logU / (1j * T) #- shift * qt.qeye(hamiltonian_qt.shape[0])
# qiskit_trott_ham_spec = np.linalg.eigvalsh(trotterized_ham_from_circuit)
# spectrum_dist = np.linalg.norm(qiskit_trott_ham_spec - exact_spec)
# print(f'Spectrum distance {spectrum_dist}')

#### Qiskit gate angle convention! RXX(-2x) = exp(x)

In [19]:
c = QuantumCircuit(2)
c.rxx(np.pi/2, 0, 1)
op = Operator(c)
print(op.data)
X = qt.sigmax()
XX = qt.tensor(X, X)
op_qt = expm(-1j*np.pi* XX / 4)
print(op_qt)


[[0.70710678+0.j         0.        +0.j         0.        +0.j
  0.        -0.70710678j]
 [0.        +0.j         0.70710678+0.j         0.        -0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        -0.70710678j 0.70710678+0.j
  0.        +0.j        ]
 [0.        -0.70710678j 0.        +0.j         0.        +0.j
  0.70710678+0.j        ]]
[[0.70710678+0.j         0.        +0.j         0.        +0.j
  0.        -0.70710678j]
 [0.        +0.j         0.70710678+0.j         0.        -0.70710678j
  0.        +0.j        ]
 [0.        +0.j         0.        -0.70710678j 0.70710678+0.j
  0.        +0.j        ]
 [0.        -0.70710678j 0.        +0.j         0.        +0.j
  0.70710678+0.j        ]]


### Compare qiskit and qutip gates in the Trotter

In [4]:
# np.random.seed(138)
num_qubits = 10
# dist_qutip_qiskit = 0.
# while dist_qutip_qiskit < 1:
step_size = np.random.rand()
step_size = 1.1
print(f'RXX angle , {step_size}')
i = np.random.randint(0, num_qubits)
j = (i + 1) % num_qubits
print(f'Sites {i, j}')

#* Qutip
XX = pad_term([qt.sigmax(), qt.sigmax()], num_qubits, i)
eXX = expm(1j*step_size*XX.full())

# XX_byhand_reversed = qt.tensor([qt.qeye(2), qt.sigmax(), qt.sigmax()])
# eXX_byhand_reversed = expm(1j*step_size*XX_byhand_reversed.full())
print('Qutip')
print(eXX)
#* Qiskit
rxx_circ = QuantumCircuit(num_qubits)
rxx_circ.rxx(-2 * step_size, i, j)
rxx_op = Operator(rxx_circ)
print('Qiskit')
print(rxx_op)

dist_qutip_qiskit = np.linalg.norm(eXX - rxx_op.data)
# dist_qutip_qiskit_reversed = np.linalg.norm(eXX_byhand_reversed - rxx_op.data)
print(f'Distance between Qutip and Qiskit RXX {dist_qutip_qiskit}')
# print(f'Distance between Qutip and Qiskit RXX reversed {dist_qutip_qiskit_reversed}')



RXX angle , 1.1
Sites (6, 7)
Qutip
[[0.45359611+0.j 0.        +0.j 0.        +0.j ... 0.        +0.j
  0.        +0.j 0.        +0.j]
 [0.        +0.j 0.45359611+0.j 0.        +0.j ... 0.        +0.j
  0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.45359611+0.j ... 0.        +0.j
  0.        +0.j 0.        +0.j]
 ...
 [0.        +0.j 0.        +0.j 0.        +0.j ... 0.45359611+0.j
  0.        +0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j ... 0.        +0.j
  0.45359611+0.j 0.        +0.j]
 [0.        +0.j 0.        +0.j 0.        +0.j ... 0.        +0.j
  0.        +0.j 0.45359611+0.j]]
Qiskit
Operator([[0.45359612+0.j, 0.        +0.j, 0.        +0.j, ...,
           0.        +0.j, 0.        +0.j, 0.        +0.j],
          [0.        +0.j, 0.45359612+0.j, 0.        +0.j, ...,
           0.        +0.j, 0.        +0.j, 0.        +0.j],
          [0.        +0.j, 0.        +0.j, 0.45359612+0.j, ...,
           0.        +0.j, 0.        +0.j, 0. 